In [170]:
from pymongo import MongoClient
import pandas as pd
from src import Mongo_connection as mc #function in src
from src import geoqueries as gq #function in src
from src import count_gq as cg #function in src

In [166]:
db, star = mc.connectCollection('datlab','starbucks')
db, sch = mc.connectCollection('datlab', 'schools')
db, air = mc.connectCollection('datlab', 'airports')
db, night = mc.connectCollection('datlab', 'nightclubs')
db, veg = mc.connectCollection('datlab', 'vegan_restaurants')
db, kinder = mc.connectCollection('datlab', 'kindergarten')

In [144]:
db, coll = mc.connectCollection('datlab','companies_seattle')

In [145]:
q = list(coll.find())

In [146]:
seattle = pd.DataFrame(q)

In [147]:
#Here I have splitted the location column in two columns for getting the coordinates in order to get the geo_queries
seattle = pd.concat([seattle.drop(['location','latitude'], axis=1), seattle['location'].apply(pd.Series)], axis=1)

In [148]:
listacoords = list(seattle['coordinates'])

In [150]:
sb250, sb500 = cg.count(star, listacoords)
air250, air500 = cg.count(air, listacoords)
sch250, sch500 = cg.count(sch, listacoords)
ni250, ni500 = cg.count(night, listacoords)
veg250, veg500 = cg.count(veg, listacoords)
kin250, kin500 = cg.count(kinder, listacoords)

In [151]:
sb250

[1, 0, 0, 5, 1, 0, 1, 5, 1, 0, 5, 1, 0, 1, 0, 0, 3, 3, 0, 0, 0]

In [152]:
facilities = pd.DataFrame(list(zip(sb250,air250,sch250,ni250,veg250,kin250)), columns =["Starbucks","Airports", "Schools","Nightclubs","Veganrests","Kindergarten"])

In [153]:
#At this point I rank the companies by the quantity of facilities they have around 250 and 500 metres. I only get the companies with more than 10 facilities around
seattle =pd.concat([seattle, facilities], axis=1)

In [154]:
seattle['250'] = seattle['Starbucks'] + seattle['Airports'] + seattle['Schools'] + seattle['Nightclubs'] + seattle['Veganrests'] + seattle['Kindergarten']

In [155]:
coldelete = ['Starbucks','Airports','Schools','Nightclubs','Veganrests','Kindergarten','_id','','description','city','type']
seattle = seattle.drop(coldelete, axis=1)

In [156]:
fac500 = pd.DataFrame(list(zip(sb500,air500,sch500,ni500,veg500,kin500)), columns=['Starbucks','Airports','Schools','Nightclubs','Veganrests','Kindergarten'])
seattle = pd.concat([seattle,fac500],axis=1)

In [158]:
seattle['500'] = seattle['Starbucks'] + seattle['Airports'] + seattle['Schools'] + seattle['Nightclubs'] + seattle['Veganrests'] + seattle['Kindergarten']

In [159]:
coldelete2 = ['Starbucks','Airports','Schools','Nightclubs','Veganrests','Kindergarten']
seattle = seattle.drop(coldelete2, axis=1)

In [161]:
seattle['rank'] = seattle['250'] + seattle['500']

In [162]:
seattle = seattle.loc[(seattle['rank']>10)]

In [164]:
seattle.sort_values(by='rank', ascending=False)

,name,category_code,number_of_employees,founded_year,total_money_raised,coordinates,250,500,rank
7,Optify,software,45,2008,$10.8M,"[-122.333359, 47.603364]",16,7.5,23.5
3,Implied Solutions,software,1,2008,$0,"[-122.3328401, 47.6042663]",14,7.5,21.5
17,Decho,web,150,2008,$0,"[-122.3326145, 47.6024156]",15,6.5,21.5
10,Z2Live,games_video,130,2009,$6.5M,"[-122.3399782, 47.6103008]",14,7.0,21.0
6,L4 Mobile,mobile,25,2008,$4.8M,"[-122.2049192, 47.676762]",18,2.0,20.0
16,GroupieGuide,web,4,2008,$0,"[-122.3374372, 47.6042792]",8,11.0,19.0
8,SlideME,mobile,,2008,$0,"[-122.3343564, 47.5992095]",5,5.5,10.5
